In [ ]:
# Install visualization libraries if missing
%pip install matplotlib seaborn ipykernel

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add current directory to path so we can import our modules
sys.path.append(os.getcwd())

from data.data_loader import DataLoader
from models.recommender import RecommenderSystem

# Initialize
loader = DataLoader()
recommender = RecommenderSystem(loader)

### 1. View the Interaction Matrix (The 'Scoreboard')
This shows how users have rated movies based on their behavior (Favorites=5, Watch Later=2, Views=1-3).

In [ ]:
# Get the raw interaction data
interactions_df = loader.get_interaction_matrix()

print(f"Total Interactions: {len(interactions_df)}")
print(f"Unique Users: {interactions_df['user_id'].nunique()}")
print(f"Unique Movies: {interactions_df['movie_id'].nunique()}")

# Show the first 10 rows
display(interactions_df.head(10))

# Visualize the distribution of scores
if not interactions_df.empty:
    plt.figure(figsize=(8, 4))
    sns.countplot(x='score', data=interactions_df)
    plt.title('Distribution of Interaction Scores')
    plt.show()

### 2. View Movie Metadata
This is the data used for Content-Based filtering (Genres, Ratings, etc.).

In [ ]:
movies_df = loader.get_movies_metadata()
display(movies_df.head())

### 3. Train the Model & Inspect Latent Features
We will train the model and then look at the hidden 'User Features' matrix.

In [ ]:
# Train the model
metrics = recommender.train()
print("Training Metrics:", metrics)

# Inspect the User-Feature Matrix (Hidden Preferences)
if recommender.user_features is not None:
    user_features_df = pd.DataFrame(
        recommender.user_features,
        index=list(recommender.user_id_map.keys()),
        columns=[f"Feature_{i+1}" for i in range(recommender.n_components)]
    )
    
    print("\nUser Latent Features (Top 5 Users):")
    display(user_features_df.head())
    
    # Visualize the heatmap of features for first 10 users
    plt.figure(figsize=(12, 6))
    sns.heatmap(user_features_df.head(10), cmap="viridis")
    plt.title("Hidden User Preferences (Latent Features)")
    plt.show()

### 4. Test Recommendations
Pick a user ID from the interaction matrix above and see what the model recommends.

In [ ]:
# Pick a user ID from the interactions dataframe
if not interactions_df.empty:
    test_user_id = interactions_df.iloc[0]['user_id']
    print(f"Generating recommendations for User ID: {test_user_id}")

    recommendations = recommender.get_recommendations(test_user_id, n_recommendations=5)
    
    # Convert to DataFrame for nice display
    recs_df = pd.DataFrame(recommendations)
    
    # Add movie titles
    if not recs_df.empty:
        # Helper to get title
        def get_title(mid):
            m = movies_df[movies_df['movie_id'] == mid]
            return m.iloc[0]['title'] if not m.empty else "Unknown"
        
        recs_df['title'] = recs_df['movie_id'].apply(get_title)
        display(recs_df[['title', 'score', 'reason', 'movie_id']])
    else:
        print("No recommendations found.")
else:
    print("No data available to generate recommendations.")